In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_translated_en.json
/kaggle/input/exist2025-all/EXIST2025_training_translated_es.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_majority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_dev.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_gold_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_gold_soft.json
/kaggle/input/

In [2]:
pip install nltk transformers torch tqdm


Note: you may need to restart the kernel to use updated packages.


In [8]:
import wandb

wandb.login(key="0c5f368f1f51fd942ec7bb3a1c74efb7bdc832d6")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mshoaibvohra (mshoaibvohra-habib-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import json
import torch
import nltk
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from tqdm import tqdm
from torch.nn.functional import softmax

# Ensure necessary NLTK components are available
nltk.download("punkt")

# Load the dataset
file_path = "/kaggle/input/exist2025-all/EXIST2025_training_translated_en.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Load Pretrained DistilBERT Model & Tokenizer (For Label Acquisition)
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# **Step 1: Extracting Units**
def extract_units(text):
    """Extracts meaningful sentence units (Facts/Sentences) from text."""
    sentences = nltk.sent_tokenize(text)
    return sentences if len(sentences) > 1 else [text]  # Ensure at least one unit

# **Step 2: Label Acquisition (Not needed since we're keeping original labels)**

# **Step 3 & 4: Sift & Employ**
augmented_data = {}
augmented_count = 0

for key, value in tqdm(data.items(), desc="Applying EASE Augmentation"):
    original_text = value["tweet"]
    extracted_units = extract_units(original_text)

    for i, unit in enumerate(extracted_units):
        aug_key = f"{key}_AUG_{i}"
        
        # Create an augmented version while keeping labels the same
        augmented_data[aug_key] = {
            "id_EXIST": f"{value['id_EXIST']}_AUG_{i}",
            "tweet": unit,
            "lang": value["lang"],
            "split": "AUG_EN",  # Change split to AUG_EN
            "number_annotators": value["number_annotators"],
            "annotators": value["annotators"],
            "gender_annotators": value["gender_annotators"],
            "age_annotators": value["age_annotators"],
            "ethnicities_annotators": value["ethnicities_annotators"],
            "study_levels_annotators": value["study_levels_annotators"],
            "countries_annotators": value["countries_annotators"],
            "labels_task1_1": value["labels_task1_1"],
            "labels_task1_2": value["labels_task1_2"],
            "labels_task1_3": value["labels_task1_3"]
        }
        augmented_count += 1

# Merge original and augmented datasets
data.update(augmented_data)

# Save Augmented Dataset
augmented_file_path = "EXIST2025_training_augmented_en.json"
with open(augmented_file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"\n✅ Data Augmentation Complete: {augmented_count} new samples added.")
print(f"Augmented dataset saved at: {augmented_file_path}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Applying EASE Augmentation: 100%|██████████| 6920/6920 [00:00<00:00, 13226.33it/s]



✅ Data Augmentation Complete: 13063 new samples added.
Augmented dataset saved at: EXIST2025_training_augmented_en.json
